In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**In this project I am gonna be learning to use BERT model to do sentiment analysis and after that I am be usng web scraping to test some reviews from a website.**


In [2]:
!pip install transformers requests beautifulsoup4 streamlit pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 74.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 73.9 MB/s eta 0:00:00:00:0100:01


In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
tokens = tokenizer.encode('bert is absolutely great', return_tensors='pt')
result = model(tokens)
result.logits
int(torch.argmax(result.logits))+1


tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

2025-05-12 04:55:39.612255: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747025739.860919      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747025739.935367      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

5

This 4 means positive, see the chart - 

1 star = Very Negative

2 stars = Negative

3 stars = Neutral

4 stars = Positive

5 stars = Very Positive

Now what we are going to do, we are going to scrape the data from a website called Yelp, which is basically a website for reviews. And we are going to scrape the data and we are going to know that whether the model is going to work on the given dataset or not from the website, so that we can make a website around this project that the common user can also use.

In [4]:
import requests
from bs4 import BeautifulSoup
import re
r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [5]:
#Now I am gonna rewrite the whole code for doing the sentiment analysis on each review to know whether the fine tuning is feasible or not
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

# Load the pre-trained sentiment analysis model and tokenizer
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

# Function to get sentiment from the model
def get_sentiment(text):
    tokens = tokenizer.encode(text, return_tensors='pt')
    result = model(tokens)
    sentiment_score = int(torch.argmax(result.logits)) + 1  # Sentiment range from 1 to 5
    return sentiment_score

# Scrape reviews from Yelp
url = 'https://www.yelp.com/biz/social-brew-cafe-pyrmont'  # Replace with your desired Yelp URL
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')

# Use a regex to find all review-related elements
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class': regex})

# Extract reviews text
reviews = [result.text for result in results]

# Apply sentiment analysis for each review
review_sentiments = []
for review in reviews:
    sentiment = get_sentiment(review)
    review_sentiments.append((review, sentiment))

# Print out reviews with sentiment scores
for review, sentiment in review_sentiments:
    print(f"Review: {review}\nSentiment Score: {sentiment}\n{'-'*50}")


Review: Very cute coffee shop and restaurant. They have a lovely outdoor seating area and several tables inside.  It was fairly busy on a Tuesday morning but we were to grab the last open table. The server was so enjoyable, she chatted and joked with us and provided fast service with our ordering, drinks and meals. The food was very good. We ordered a wide variety and every meal was good to delicious. The sweet potato fries on the Chicken Burger plate were absolutely delicious, some of the best I've ever had. I definitely enjoyed this cafe, the outdoor seating, the service and the food!!
Sentiment Score: 5
--------------------------------------------------
Review: Amazing ambience. Coffee was the best we've had in Sydney. The service and food also great. As must stop for breakfast.
Sentiment Score: 5
--------------------------------------------------
Review: Six of us met here for breakfast before our walk to Manly. We were enjoying visiting with each other so much that I apologize for

Making the web app to make the app feasible

In [ ]:
import streamlit as st
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import os
import subprocess
import time
from pyngrok import ngrok

# ========== ENVIRONMENT ==========
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# ========== MODEL ==========
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

def get_sentiment_label(score):
    return {
        1: "Very Negative",
        2: "Negative",
        3: "Neutral",
        4: "Positive",
        5: "Very Positive"
    }.get(score, "Unknown")

def get_sentiment(text):
    tokens = tokenizer.encode(text, return_tensors='pt')
    result = model(tokens)
    score = int(torch.argmax(result.logits)) + 1
    label = get_sentiment_label(score)
    return score, label

# ========== STREAMLIT APP CODE ==========
with open("app.py", "w") as f:
    f.write("""
import streamlit as st
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"

tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

def get_sentiment_label(score):
    return {
        1: "Very Negative",
        2: "Negative",
        3: "Neutral",
        4: "Positive",
        5: "Very Positive"
    }.get(score, "Unknown")

def get_sentiment(text):
    tokens = tokenizer.encode(text, return_tensors='pt')
    result = model(tokens)
    score = int(torch.argmax(result.logits)) + 1
    label = get_sentiment_label(score)
    return score, label

st.title("Sentimeter")
review = st.text_area("Enter a review:")

if st.button("Find"):
    if review:
        score, label = get_sentiment(review)
        # Color-based output based on sentiment label
        if label == "Very Positive" or label == "Positive":
            st.markdown(f'<div style="background-color: green; padding: 10px; color: white; border-radius: 5px;">{label}</div>', unsafe_allow_html=True)
        elif label == "Negative" or label == "Very Negative":
            st.markdown(f'<div style="background-color: #D22B2B; padding: 10px; color: white; border-radius: 5px;">{label}</div>', unsafe_allow_html=True)
        else:
            st.write(f"{label}")  # For Neutral, just plain text
    else:
        st.warning("Please enter a review.")
    """)

# ========== NGROK SETUP ==========
ngrok.kill()  # kill any previous sessions
ngrok.set_auth_token(" replace with your real token")  

# Start the Streamlit app
process = subprocess.Popen(["streamlit", "run", "app.py"])
time.sleep(5)  # wait for streamlit to boot up

# Get public URL from ngrok
public_url = ngrok.connect(8501)
print("🚀 App running at:", public_url)

# Keep it alive (manually stop from "Stop Execution" button in Kaggle)
process.wait()





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.19.2.2:8501
  External URL: http://34.141.218.43:8501

🚀 App running at: NgrokTunnel: "https://d957-34-141-218-43.ngrok-free.app" -> "http://localhost:8501"


2025-05-12 05:51:08.092261: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747029068.125784     218 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747029068.135717     218 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-12 05:51:12.762 Examining the path of torch.classes raised:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/streamlit/web/bootstrap.py", line 347, in run
    if asyncio.get_running_loop().is_running():
       ^^^^^^^^^^^^^^^^^^^^^^^^^^
RuntimeError: no running event loop

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/loca